<a href="https://colab.research.google.com/github/JWHer/BERT/blob/main/imsdb/IMSDB_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
#https://medium.com/@eyfydsyd97/bert%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%9C-classification-by-pytorch-2a6d4adaf162

     |████████████████████████████████| 1.4MB 19.6MB/s 
     |████████████████████████████████| 2.9MB 60.5MB/s 
     |████████████████████████████████| 890kB 55.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=dc14bc313536721abc28c0d0f18cb675d8f5dbf60294335870eb61854435ffd7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import gc

import transformers
from transformers import BertTokenizer
from transformers import BertModel

# 하이퍼파라미터
Max_len=512
Batch_size=8
num_category = 8
lr = 3e-5
EPOCHS=3

In [5]:
#Load data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Check path
PATH='/content/drive/My Drive/datas/'
filenames=os.listdir(PATH)
print(filenames)

['186_Pirates of the Caribbean.csv', '51_conan_the_barbarian.csv', '229_Surrogates.csv', '230_Swordfish.csv', '231_Terminator.csv', 'last.pt', 'best.pt', 'acc.csv']


In [7]:
# 데이터
train=pd.read_csv(PATH+"231_Terminator.csv")
print(train)
train_text = train['sentence']
train_label = train['label']

valid=pd.read_csv(PATH+"51_conan_the_barbarian.csv")
print(valid)
valid_text = valid['sentence']
valid_label = valid['label']
print("\ndata loaded\n")

                                               sentence  label
0                            " T E R M I N A T O R " by      0
1                                         James Cameron      0
2                                       Registered WGAw      0
3                                          Fourth Draft      0
4                                             20.Apr.83      0
...                                                 ...    ...
4233                                              SARAH      2
4234                                          (quietly)      4
4235                                            I know.      3
4236  CAMERA CRANES UP as she pulls away, driving ac...      1
4237  A brilliant flash crescendos from horizon to h...      1

[4238 rows x 2 columns]
                                               sentence  label
0     In the darkness, we hear the solitary sound of...      1
1                                              NARRATOR      2
2     In between the years whe

In [8]:
# 토크나이저
tokenizer= BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

def text_processing(text, Max_len):
  input_ids = []
  mask_ids = []
  token_type_ids = []
  for line in text:
    # [cls], [sep] 추가 Max_len 길이 맞추기  
    encoded_line = tokenizer.encode(line, add_special_tokens = True, max_length=Max_len, truncation=True) 
    input = encoded_line + [0]*(Max_len-len(encoded_line))
    token_type =  [0]*len(input)
    mask = [1]*len(encoded_line)+[0]*(Max_len-len(encoded_line))    
    input_ids.append(torch.tensor(input))
    token_type_ids.append(torch.tensor(token_type))
    mask_ids.append(torch.tensor(mask))

  return input_ids, token_type_ids, mask_ids
  
train_input_ids, train_token_type_ids, train_mask_ids =text_processing(train_text, Max_len)
valid_input_ids, valid_token_type_ids, valid_mask_ids =text_processing(valid_text, Max_len)  

In [9]:
# 데이터로더
class BertCls_dataset(Dataset):
  def __init__(self, input_ids, token_type_ids, mask_ids, label):
    self.input = input_ids
    self.token_type = token_type_ids
    self.attn_mask = mask_ids
    self.label = label
  def __len__(self):
    return len(self.input)  

  def __getitem__(self, idx):
    x = self.input[idx]
    y = self.token_type[idx]
    z = self.attn_mask[idx]
    label = self.label[idx]
    label = [label]

    return x, y, z ,torch.tensor(label)

train_dataset=BertCls_dataset(train_input_ids, train_token_type_ids, train_mask_ids, train_label)
valid_dataset=BertCls_dataset(valid_input_ids, valid_token_type_ids, valid_mask_ids, valid_label)

train_dataloader = DataLoader(train_dataset, batch_size=Batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=Batch_size, shuffle=True)


In [10]:
# 모델
bertcls = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True)

class BertCls(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert_cls = bertcls

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.3)
    self.out = nn.Linear(768*2, num_category)

  def forward(self, input_ids, token_type_ids, mask_ids):
    #print(input_ids, mask_ids, token_type_ids)
    outputs = self.bert_cls(input_ids, attention_mask=mask_ids, token_type_ids=token_type_ids)
    #print("output: ", end="")
    #print(outputs.last_hidden_state)
    output=outputs.last_hidden_state

    apool = torch.mean(output, 1)
    mpool, _ = torch.max(output, 1)
    x = torch.cat((apool, mpool), 1) #[cls]를 사용하는 방식 대신, mean, max를 한 후 concat한 output을 사용하였다.
    x = self.relu(x)
    x = self.dropout(x)
    x = self.out(x)
    return x

  def save(self, path):
    torch.save(self.state_dict(), path)

  def load(self, path):
    self.load_state_dict(torch.load(path))

  def predict(self, strList):

    input_ids, token_type_ids, mask_ids =text_processing(strList, Max_len)

    input_ids = input_ids[0].resize_(1, len(input_ids[0])).to(DEVICE, dtype=torch.long)
    token_type_ids = token_type_ids[0].resize_(1,len(token_type_ids[0])).to(DEVICE, dtype=torch.long)
    mask_ids=mask_ids[0].resize_(1,len(mask_ids[0])).to(DEVICE, dtype=torch.long)

    # label=[0]*len(strList)
    # dataset=BertCls_dataset(input_ids, token_type_ids, mask_ids, label)

    # input_ids, token_type_ids, attn_mask_ids, label = dataset[0]
    # input_ids = input_ids.to(DEVICE, dtype=torch.long)
    # token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
    # attn_mask_ids = attn_mask_ids.to(DEVICE, dtype=torch.long)
    #label = label.to(DEVICE, dtype=torch.long)

    with torch.no_grad():
      output = model(input_ids, token_type_ids, mask_ids)
    return torch.argmax(output[0]).item()

In [11]:
# 학습
def train_one_epoch(data_loader, model, optimizer, device, loss_fn):
  
  model.train()
  tk0 = tqdm(data_loader, total=len(data_loader))
  total_loss = 0.0
  
  for bi, d in enumerate(tk0):
      input_ids, token_type_ids, attn_mask_ids, label = d
      input_ids = input_ids.to(device, dtype=torch.long)
      token_type_ids = token_type_ids.to(device, dtype=torch.long)
      attn_mask_ids = attn_mask_ids.to(device, dtype=torch.long)
      label = label.to(device, dtype=torch.long)

      #model.zero_grad()
      output = model(input_ids, token_type_ids, attn_mask_ids)
      loss = loss_fn(output, label.view(-1))
      total_loss += loss.item()
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      del output
      torch.cuda.empty_cache()

  del tk0
  avg_train_loss = total_loss / len(data_loader) 
  print(" Average training loss: {0:.2f}".format(avg_train_loss))  
  return avg_train_loss

def eval_one_epoch(data_loader, model,  device, loss_fn):

  model.eval()
  tk0 = tqdm(data_loader, total=len(data_loader))
  fin_targets = []
  fin_outputs = []
  
  with torch.no_grad():

    for bi, d in enumerate(tk0):
      input_ids, token_type_ids, attn_mask_ids, label = d
      input_ids = input_ids.to(device, dtype=torch.long)
      token_type_ids = token_type_ids.to(device, dtype=torch.long)
      attn_mask_ids = attn_mask_ids.to(device, dtype=torch.long)
      label = label.to(device, dtype=torch.long)

      output = model(input_ids, token_type_ids, attn_mask_ids)
      loss = loss_fn(output, label.view(-1))

      output = output.detach().cpu().numpy()
      label = label.detach().cpu().numpy()
      pred = np.argmax(output, axis=1).flatten()

      fin_targets.extend(label.tolist())
      fin_outputs.extend(pred.tolist()) 

      del output
      torch.cuda.empty_cache()
  del tk0
    
  return fin_outputs, fin_targets
  
def fit(train_dataloader, valid_dataloader, EPOCHS=3):
  Bert_cls=BertCls() #model
  best_auc=None
  if "best.pt" in os.listdir(PATH):
    Bert_cls.load(PATH+"best.pt")
    acc_df=pd.read_csv(PATH+"acc.csv")
    best_auc=acc_df['acc']
    print("Load from best model. ACC {}".format(best_auc))
    
  Bert_cls=Bert_cls.to(DEVICE)
  loss_fn = nn.CrossEntropyLoss() #loss
  optimizer = torch.optim.AdamW(Bert_cls.parameters(),lr=lr) #optimizer

  for i in range(EPOCHS):
    print(f"EPOCHS:{i+1}")
    print('TRAIN')
    loss = train_one_epoch(train_dataloader, Bert_cls, optimizer, DEVICE, loss_fn)

    print('EVAL')
    outputs, targets = eval_one_epoch(valid_dataloader, Bert_cls,  DEVICE, loss_fn)
    targets = np.array(targets)
    auc = accuracy_score(targets, outputs)
    print(f"auc;{auc}") 

    if best_auc==None or best_auc<auc:
      print("Save model...")
      Bert_cls.save(PATH+"best.pt")
      acc_df = pd.DataFrame({"acc":auc, "loss":loss}, index=[0])
      acc_df.to_csv(PATH+"acc.csv")
      best_auc=auc

  return Bert_cls

In [ ]:
# 학습
print("Making model...")

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    gc.collect()
    torch.cuda.empty_cache()
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    DEVICE = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

model = fit(train_dataloader, valid_dataloader)

In [12]:
model=BertCls()
model.load(PATH+"best.pt")
# DEVICE="cuda"
# model.to(DEVICE)

tag=["기타", "상황", "인물", "대사", "묘사", "장소", "시간", ]

In [13]:
prediction= model.predict(['JOHN'])
print(tag[prediction])

prediction=model.predict(['The sound of a man totally cracking up.'])
print(tag[prediction])

prediction=model.predict(['is it just me, or is it getting crazier out there?'])
print(tag[prediction])

prediction=model.predict(['INT. DEPT. OF HEALTH, OFFICE - MORNING'])
print(tag[prediction])

인물
상황
대사
장소
